In [1]:
"""
OPTIMIZED RANGE BACKTEST - TODAY ONLY
10:00-11:15 AM RANGE (75 minutes)
20 Contracts | 10pt SL | 100pt TP (10:1 R/R)
✅ Includes realistic slippage & commissions
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import pytz

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# OPTIMIZED Strategy parameters (from 31-day backtest)
POSITION_SIZE = 20
FIXED_STOP_LOSS = 10
FIXED_TAKE_PROFIT = 100

# Realism parameters
SLIPPAGE_POINTS = 0.25  # 1 tick for MNQ
COMMISSION_PER_CONTRACT = 0.62  # Per side
POINT_VALUE = 0.5  # $0.50 per point for MNQ

# Range parameters
RANGE_START_HOUR = 10
RANGE_START_MINUTE = 0
RANGE_END_HOUR = 11
RANGE_END_MINUTE = 15

# ==================== API FUNCTIONS ====================
def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def fetch_bars(headers, unit, unit_number):
    end_time = datetime.now(pytz.UTC)
    start_time = end_time - timedelta(days=2)
    
    payload = {
        "contractId": CONTRACT_ID,
        "live": False,
        "startTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "unit": unit,
        "unitNumber": unit_number,
        "limit": 10000,
        "includePartialBar": True
    }
    
    resp = requests.post(f"{API_BASE}/History/retrieveBars", json=payload, headers=headers, timeout=30)
    return resp.json().get("bars", [])

def parse_bars_with_ny_time(bars):
    ny_tz = pytz.timezone('America/New_York')
    parsed = []
    
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar['dt_ny'] = dt_ny
        bar['date'] = dt_ny.date()
        parsed.append(bar)
    
    return parsed

def get_morning_range(bars_5m, today):
    """Get the range from 10:00 AM - 11:15 AM using 5-minute candles"""
    print(f"\n🔍 DEBUG: All 5-min bars for today {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d}:")
    
    # Find all 5-min bars in the range window
    morning_bars = []
    for bar in bars_5m:
        if bar['date'] == today:
            bar_time_minutes = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            start_time_minutes = RANGE_START_HOUR * 60 + RANGE_START_MINUTE
            end_time_minutes = RANGE_END_HOUR * 60 + RANGE_END_MINUTE
            
            if start_time_minutes <= bar_time_minutes < end_time_minutes:
                morning_bars.append(bar)
                print(f"   {bar['dt_ny'].strftime('%I:%M %p')} - H: {bar['h']:.2f}, L: {bar['l']:.2f}")
    
    if not morning_bars:
        return None
    
    # Calculate the range high and low
    range_high = max(bar['h'] for bar in morning_bars)
    range_low = min(bar['l'] for bar in morning_bars)
    
    print(f"\n✅ Morning Range ({RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d} - {RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d}):")
    print(f"   High: {range_high:.2f}")
    print(f"   Low: {range_low:.2f}")
    print(f"   Size: {range_high - range_low:.2f} points")
    
    return {
        'high': range_high,
        'low': range_low
    }

def simulate_trade_realistic(direction, entry, stop, target, future_bars):
    """
    Realistic trade simulation with slippage, commissions, and intra-bar logic
    """
    # Apply slippage to entry
    if direction == 'long':
        entry_with_slippage = entry + SLIPPAGE_POINTS
        stop_with_slippage = stop - SLIPPAGE_POINTS
        target_with_slippage = target - SLIPPAGE_POINTS
    else:
        entry_with_slippage = entry - SLIPPAGE_POINTS
        stop_with_slippage = stop + SLIPPAGE_POINTS
        target_with_slippage = target + SLIPPAGE_POINTS
    
    for bar in future_bars:
        if direction == 'long':
            stop_hit = bar['l'] <= stop_with_slippage
            target_hit = bar['h'] >= target_with_slippage
            
            # If both hit in same bar, check which was closer
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (stop_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (target_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
        
        else:  # SHORT
            stop_hit = bar['h'] >= stop_with_slippage
            target_hit = bar['l'] <= target_with_slippage
            
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (entry_with_slippage - stop_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (entry_with_slippage - target_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
    
    # Trade still open at end of day
    if future_bars:
        exit_price = future_bars[-1]['c']
        if direction == 'long':
            gross_pnl = (exit_price - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
        else:
            gross_pnl = (entry_with_slippage - exit_price) * POINT_VALUE * POSITION_SIZE
        
        commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
        net_pnl = gross_pnl - commissions
        return {'result': 'OPEN', 'pnl': net_pnl, 'exit_price': exit_price}
    
    return {'result': 'OPEN', 'pnl': 0, 'exit_price': entry_with_slippage}

# ==================== BACKTEST TODAY ====================
def backtest_today(bars_5m, today):
    trades = []
    
    # Get morning range
    range_data = get_morning_range(bars_5m, today)
    
    if range_data is None:
        return None, "No morning range found for today"
    
    range_high = range_data['high']
    range_low = range_data['low']
    
    # Get today's 5-minute bars AFTER range ends
    bars_today = []
    for b in bars_5m:
        if b['date'] == today:
            bar_time_minutes = b['dt_ny'].hour * 60 + b['dt_ny'].minute
            range_end_minutes = RANGE_END_HOUR * 60 + RANGE_END_MINUTE
            
            if bar_time_minutes >= range_end_minutes:
                bars_today.append(b)
    
    # IMPORTANT: Reverse to chronological order (oldest first)
    bars_today = list(reversed(bars_today))
    
    if len(bars_today) < 10:
        return None, f"Not enough 5m bars after {RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d}"
    
    # Scan for setups
    print(f"\nScanning {len(bars_today)} bars for setups (after {RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d})...")
    print(f"Range: {range_high:.2f} (high) - {range_low:.2f} (low)")
    print(f"Range Size: {range_high - range_low:.2f} points\n")
    print("Showing first 20 bars after range:")
    print("-" * 80)
    
    for i in range(min(20, len(bars_today))):
        bar = bars_today[i]
        status = ""
        if bar['c'] > range_high:
            status = "ABOVE range"
        elif bar['c'] < range_low:
            status = "BELOW range"
        else:
            status = "INSIDE range"
        
        print(f"{bar['dt_ny'].strftime('%I:%M %p')} - Close: {bar['c']:.2f} ({status})")
    
    print("-" * 80)
    print("\nNow checking for setups...\n")
    
    for i in range(len(bars_today) - 1):
        current_bar = bars_today[i]
        next_bar = bars_today[i + 1]
        
        # Step 1: Check if current bar closed OUTSIDE range
        closed_above = current_bar['c'] > range_high
        closed_below = current_bar['c'] < range_low
        
        # Step 2: Check if NEXT bar closed back INSIDE range
        next_closed_inside = range_low <= next_bar['c'] <= range_high
        
        # Debug: Print when we find potential setups
        if closed_above and next_closed_inside:
            print(f"[FOUND SHORT SETUP]")
            print(f"  Breakout Bar: {current_bar['dt_ny'].strftime('%I:%M %p')} - Close: {current_bar['c']:.2f} (ABOVE {range_high:.2f})")
            print(f"  Entry Bar: {next_bar['dt_ny'].strftime('%I:%M %p')} - Close: {next_bar['c']:.2f} (back INSIDE)")
            print()
        
        if closed_below and next_closed_inside:
            print(f"[FOUND LONG SETUP]")
            print(f"  Breakout Bar: {current_bar['dt_ny'].strftime('%I:%M %p')} - Close: {current_bar['c']:.2f} (BELOW {range_low:.2f})")
            print(f"  Entry Bar: {next_bar['dt_ny'].strftime('%I:%M %p')} - Close: {next_bar['c']:.2f} (back INSIDE)")
            print()
        
        # SHORT SETUP: Broke above, came back in
        if closed_above and next_closed_inside:
            entry_price = next_bar['c']
            stop_loss = entry_price + FIXED_STOP_LOSS
            take_profit = entry_price - FIXED_TAKE_PROFIT
            
            future_bars = bars_today[i+2:]
            outcome = simulate_trade_realistic('short', entry_price, stop_loss, take_profit, future_bars)
            
            trades.append({
                'time': next_bar['dt_ny'],
                'type': 'SHORT',
                'entry': entry_price,
                'stop': stop_loss,
                'target': take_profit,
                'result': outcome['result'],
                'pnl': outcome['pnl'],
                'exit_price': outcome['exit_price'],
                'breakout_bar_time': current_bar['dt_ny'],
                'entry_bar_time': next_bar['dt_ny']
            })
        
        # LONG SETUP: Broke below, came back in
        elif closed_below and next_closed_inside:
            entry_price = next_bar['c']
            stop_loss = entry_price - FIXED_STOP_LOSS
            take_profit = entry_price + FIXED_TAKE_PROFIT
            
            future_bars = bars_today[i+2:]
            outcome = simulate_trade_realistic('long', entry_price, stop_loss, take_profit, future_bars)
            
            trades.append({
                'time': next_bar['dt_ny'],
                'type': 'LONG',
                'entry': entry_price,
                'stop': stop_loss,
                'target': take_profit,
                'result': outcome['result'],
                'pnl': outcome['pnl'],
                'exit_price': outcome['exit_price'],
                'breakout_bar_time': current_bar['dt_ny'],
                'entry_bar_time': next_bar['dt_ny']
            })
    
    return trades, (range_high, range_low)

# ==================== MAIN ====================
def main():
    print("=" * 70)
    print("OPTIMIZED RANGE BACKTEST - TODAY ONLY")
    print(f"Range: {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d} (75 min)")
    print(f"Setup: 20 Contracts | {FIXED_STOP_LOSS}pt SL | {FIXED_TAKE_PROFIT}pt TP (10:1 R/R)")
    print(f"Realistic: Slippage {SLIPPAGE_POINTS}pts | Commission ${COMMISSION_PER_CONTRACT*2:.2f}/contract")
    print("=" * 70)
    
    # Get today's date
    ny_tz = pytz.timezone('America/New_York')
    today = datetime.now(ny_tz).date()
    
    print(f"\n📅 Testing: {today}")
    
    print("\n[1/3] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/3] Fetching data...")
    bars_5m = fetch_bars(headers, unit=2, unit_number=5)
    print(f"✅ Fetched {len(bars_5m)} 5m bars")
    
    bars_5m = parse_bars_with_ny_time(bars_5m)
    
    print("\n[3/3] Running backtest...")
    trades, range_or_error = backtest_today(bars_5m, today)
    
    if trades is None:
        print(f"❌ {range_or_error}")
        return
    
    range_high, range_low = range_or_error
    
    print("✅ Backtest complete")
    
    print("\n" + "=" * 70)
    print("RESULTS")
    print("=" * 70)
    
    print(f"\n📊 {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d} RANGE:")
    print(f"   HIGH: {range_high:.2f}")
    print(f"   LOW:  {range_low:.2f}")
    print(f"   SIZE: {range_high - range_low:.2f} points")
    
    if not trades:
        print("\n⚠️ No valid setups found today")
        print("   (No breakout + re-entry occurred)")
        return
    
    # Calculate stats
    wins = [t for t in trades if t['result'] == 'WIN']
    losses = [t for t in trades if t['result'] == 'LOSS']
    open_trades = [t for t in trades if t['result'] == 'OPEN']
    
    total_pnl = sum(t['pnl'] for t in trades)
    
    print(f"\n📈 TRADES: {len(trades)}")
    print(f"   Wins: {len(wins)}")
    print(f"   Losses: {len(losses)}")
    print(f"   Still Open: {len(open_trades)}")
    
    print(f"\n💰 TOTAL P&L: ${total_pnl:.2f} (after slippage & commissions)")
    if len(trades) > 0:
        print(f"   Expectancy: ${total_pnl/len(trades):.2f} per trade")
    
    print("\n" + "=" * 70)
    print("ALL TRADES")
    print("=" * 70)
    
    for trade in trades:
        if trade['result'] == 'WIN':
            symbol = "✅"
        elif trade['result'] == 'LOSS':
            symbol = "❌"
        else:
            symbol = "⏳"
        
        print(f"\n{symbol} {trade['type']} @ {trade['time'].strftime('%I:%M %p')}")
        print(f"   Breakout: {trade['breakout_bar_time'].strftime('%I:%M %p')} (closed outside)")
        print(f"   Entry: {trade['entry_bar_time'].strftime('%I:%M %p')} (closed back in)")
        print(f"   Entry Price: {trade['entry']:.2f} | Exit: {trade['exit_price']:.2f}")
        print(f"   Stop: {trade['stop']:.2f} | Target: {trade['target']:.2f}")
        print(f"   Result: {trade['result']}")
        print(f"   P&L: ${trade['pnl']:.2f}")
    
    print("\n" + "=" * 70)
    print("✅ DONE")
    print("=" * 70)
    print("\nNote: This uses the OPTIMIZED parameters from 31-day backtest:")
    print(f"  • Best performing range window: {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d}")
    print(f"  • Best TP/SL combo: {FIXED_STOP_LOSS}pt SL / {FIXED_TAKE_PROFIT}pt TP")
    print(f"  • Historical expectancy: $76.27 per trade")
    print(f"  • Historical win rate: 15.9%")
    print(f"  • Historical profit factor: 1.42")

if __name__ == "__main__":
    main()

OPTIMIZED RANGE BACKTEST - TODAY ONLY
Range: 10:00-11:15 (75 min)
Setup: 20 Contracts | 10pt SL | 100pt TP (10:1 R/R)
Realistic: Slippage 0.25pts | Commission $1.24/contract

📅 Testing: 2026-02-02

[1/3] Authenticating...
✅ Authenticated

[2/3] Fetching data...
✅ Fetched 215 5m bars

[3/3] Running backtest...

🔍 DEBUG: All 5-min bars for today 10:00-11:15:
   11:10 AM - H: 25857.50, L: 25841.50
   11:05 AM - H: 25850.50, L: 25803.25
   11:00 AM - H: 25844.75, L: 25813.75
   10:55 AM - H: 25859.00, L: 25816.25
   10:50 AM - H: 25873.50, L: 25838.00
   10:45 AM - H: 25875.50, L: 25844.25
   10:40 AM - H: 25906.50, L: 25863.00
   10:35 AM - H: 25907.25, L: 25866.25
   10:30 AM - H: 25887.50, L: 25851.50
   10:25 AM - H: 25906.25, L: 25863.50
   10:20 AM - H: 25912.25, L: 25883.75
   10:15 AM - H: 25905.25, L: 25856.75
   10:10 AM - H: 25878.00, L: 25815.50
   10:05 AM - H: 25836.25, L: 25789.75
   10:00 AM - H: 25798.00, L: 25714.00

✅ Morning Range (10:00 - 11:15):
   High: 25912.25
   L